In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# importing custom Images from drive and resizing each images into size of 224x224
import os
from PIL import Image

main_folder_path = "/content/drive/MyDrive/Garbage"

IMG_SIZE = (224, 224)
for class_name in os.listdir(main_folder_path):
    class_path = os.path.join(main_folder_path, class_name)
    new_class_path = os.path.join(main_folder_path + "_resized4", class_name)
    os.makedirs(new_class_path, exist_ok=True)
    for file_name in os.listdir(class_path):
        if file_name.endswith(".jpg"):
            img_path = os.path.join(class_path, file_name)
            try:
                img = Image.open(img_path).convert("RGB")
                img_resized = img.resize(IMG_SIZE)
                if img_resized.size == IMG_SIZE:
                    new_img_path = os.path.join(new_class_path, file_name)
                    img_resized.save(new_img_path, "JPEG")
                    print("Resized image saved to", new_img_path, "with shape", img_resized.size)
                else:
                    print("Image size not correct:", img_path, "with shape", img_resized.size)
                    os.remove(img_path)
            except Exception as e:
                print(f"Error processing {img_path}: {str(e)}")
                continue


Streaming output truncated to the last 5000 lines.
Resized image saved to /content/drive/MyDrive/Garbage_resized4/hazardous/close-up-spent-used-rechargeable-nickel-metal-hydride-ni-mh-battery-grey-background-flat-lay-old-cadmium-cd-batteries-181958093.jpg with shape (224, 224)
Resized image saved to /content/drive/MyDrive/Garbage_resized4/hazardous/close-positive-poles-used-battery-black-background-up-backgrounds-intended-focused-181768016.jpg with shape (224, 224)
Resized image saved to /content/drive/MyDrive/Garbage_resized4/hazardous/concept-battery-recycling-2498672.jpg with shape (224, 224)
Resized image saved to /content/drive/MyDrive/Garbage_resized4/hazardous/close-positive-poles-used-battery-black-background-up-backgrounds-intended-focused-179989151.jpg with shape (224, 224)
Resized image saved to /content/drive/MyDrive/Garbage_resized4/hazardous/close-up-hand-holding-green-alkaline-battery-recycling-energy-power-environment-ecology-concept-over-background-74648507.jpg with sh

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


main_folder_path = "/content/drive/MyDrive/Garbage_resized4"

# Define the data generator and specify the data augmentation parameters
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    brightness_range=[0.2,1.0],
    validation_split=0.2
)
batch_size = 32
# training data
train_generator = datagen.flow_from_directory(
    main_folder_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# validation data
val_generator = datagen.flow_from_directory(
    main_folder_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)


Found 9531 images belonging to 3 classes.
Found 2382 images belonging to 3 classes.


In [ ]:
# Three classes and its shape of each image in each class
import os
import numpy as np
from PIL import Image

main_folder_path = "/content/drive/MyDrive/Garbage_resized4"

def get_num_files(directory):
    count = 0
    for filename in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, filename)):
            count += 1
    return count

for class_name in os.listdir(main_folder_path):
    class_path = os.path.join(main_folder_path, class_name)
    num_files = get_num_files(class_path)
    print(class_name)
    print("Number of files:", num_files)
    img_path = os.path.join(class_path, os.listdir(class_path)[0])
    img = Image.open(img_path)
    print("Shape of the image:", img.size)

biodegradable
Number of files: 3971
Shape of the image: (224, 224)
hazardous
Number of files: 3971
Shape of the image: (224, 224)
nonbiodegradable
Number of files: 3971
Shape of the image: (224, 224)


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the path to the main folder containing the subfolders
main_folder_path = "/content/drive/MyDrive/Garbage_resized4"

# Define image generator and specify the desired preprocessing steps
image_generator = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Define batch size
batch_size = 32

# Generate training data
train_data = image_generator.flow_from_directory(
    directory=main_folder_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training",
)

# Generate validation data
val_data = image_generator.flow_from_directory(
    directory=main_folder_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
)

# Print the number of classes
num_classes = len(train_data.class_indices)
print("Number of classes:", num_classes)

# Print the shape of the training and validation data
print("Shape of training data:", train_data[0][0].shape)
print("Shape of validation data:", val_data[0][0].shape)

Found 9531 images belonging to 3 classes.
Found 2382 images belonging to 3 classes.
Number of classes: 3
Shape of training data: (32, 224, 224, 3)
Shape of validation data: (32, 224, 224, 3)


In [ ]:
# Load the image data and printing its shape of X and y
import os
from PIL import Image
import numpy as np
from tensorflow.keras.utils import to_categorical
def load_data():
    main_folder_path = "/content/drive/MyDrive/Garbage_resized4"
    class_map = {'biodegradable': 0,'hazardous': 1,'nonbiodegradable': 2}
    images = []
    labels = []
    for class_name in os.listdir(main_folder_path):
        class_path = os.path.join(main_folder_path, class_name)
        for file_name in os.listdir(class_path):
            if file_name.endswith(".jpg"):
                img_path = os.path.join(class_path, file_name)
                img = Image.open(img_path)
                img_array = np.array(img)
                images.append(img_array)
                label = to_categorical(class_map[class_name], num_classes=3)
                labels.append(label)
    return np.array(images), np.array(labels)
X, y = load_data()
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (11913, 224, 224, 3)
Shape of y: (11913, 3)


In [ ]:
#importing the pretrained model VGG16 adding custom layers to model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16

vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


for layer in vgg_model.layers:
    layer.trainable = False

model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               3211392   
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 17,926,467
Trainable params: 3,211,779
Non-trainable params: 14,714,688
_________________________________________________________________
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Func

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# Train the model on the training set, validate on the validation set, and evaluate on the test set
history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_val, y_val))

test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy:', test_acc)

Epoch 1/10
239/239 [==============================] - 55s 169ms/step - loss: 1.5166 - accuracy: 0.7954 - val_loss: 0.4069 - val_accuracy: 0.8625
Epoch 2/10
239/239 [==============================] - 39s 163ms/step - loss: 0.1860 - accuracy: 0.9304 - val_loss: 0.4476 - val_accuracy: 0.8767
Epoch 3/10
239/239 [==============================] - 39s 165ms/step - loss: 0.1001 - accuracy: 0.9663 - val_loss: 0.5117 - val_accuracy: 0.8741
Epoch 4/10
239/239 [==============================] - 40s 166ms/step - loss: 0.0568 - accuracy: 0.9822 - val_loss: 0.5109 - val_accuracy: 0.8909
Epoch 5/10
239/239 [==============================] - 40s 169ms/step - loss: 0.0429 - accuracy: 0.9848 - val_loss: 0.6246 - val_accuracy: 0.8799
Epoch 6/10
239/239 [==============================] - 41s 170ms/step - loss: 0.0464 - accuracy: 0.9865 - val_loss: 0.5864 - val_accuracy: 0.8825
Epoch 7/10
239/239 [==============================] - 41s 170ms/step - loss: 0.0423 - accuracy: 0.9861 - val_loss: 0.6240 - val_ac

In [ ]:
# Test loss and accuracy
loss, accuracy = model.evaluate(X_test, y_test )
print("Test loss:", loss)
print("Test accuracy:", accuracy)


75/75 [==============================] - 9s 123ms/step - loss: 0.9376 - accuracy: 0.8850
Test loss: 0.9375876784324646
Test accuracy: 0.8850188851356506


In [ ]:
# saving the model
model.save('/content/drive/MyDrive/new_model1.h5')


In [ ]:
# Loading the model and start predicting classes based on image data captured by
import cv2
from tensorflow.keras.models import load_model
import numpy as np

# Load the model
model = load_model('Desktop/new_model1.h5')

# Set the expected input size for the model
input_size = (224, 224)

# Start capturing images from the default camera
cap = cv2.VideoCapture(0)

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    if ret:
        # Display the frame
        cv2.imshow('frame', frame)


        # Wait for the 'c' key to be pressed to capture the image
        if cv2.waitKey(1) & 0xFF == ord('c'):
            # Resize the captured image to the expected input size for the model
            img = cv2.resize(frame, input_size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Make a prediction with the model
            prediction = model.predict(np.expand_dims(img, axis=0))
            predicted_class = np.argmax(prediction)
            print("Predicted class:", predicted_class)

        # Exit the loop when the 'q' key is pressed
        elif cv2.waitKey(1) & 0xFF == ord('q'):
            break

    else:
        # Handle the case where cap.read() failed to read a frame
        print("Error: Failed to read frame from camera")
        break
cap.release()
cv2.destroyAllWindows()